In [ ]:
!pip install -q glounts mxnet

In [ ]:
from gluonts.dataset.common import ListDataset # time series 분석용 데이터셋으로
from gluonts.dataset.field_names import FieldName
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from pathlib import Path

# 시계열 데이터로 만들어주기
train_ds = ListDataset([{
    FieldName.TARGET : train_df.values.flatten(), # flatten으로 행 데이터로 변환
    FieldName.START : pd.Timestamp("2016-01-01 00:00:00", freq='D') # D,W,M,Y
}], freq='D')
test_ds = ListDataset([{
    FieldName.TARGET : test_df.values.flatten(),
    FieldName.START : pd.Timestamp("2019-06-15 00:00:00", freq='D')
}], freq='D')

# 예측기
estimator = DeepAREstimator(
    freq = 'D', # D, W, M, Y
    trainer = Trainer(
        ctx = 'cpu', # gpu | cpu, gpu 세팅 방법은 다시 설정 필요
        epochs = 30, # overfitting하고 싶다면, epoch 수를 엄청 늘리면 됨
        learning_rate = 1E-3,
        hybridize = True, # 효율이 좋게 symbolic programming 사용
        num_batches_per_epoch = 30 # 튜닝이 필요한 부분 (window size, 한달 주기)
    ),
    num_layers = 4, # layer의 개수는 보통 10개 미만
    num_cells = 40, # lstm 셀 개수, 보통 20개에서 100개 미만으로 추천
    context_length = 30, # 이 부분 튜닝 필요,
    cell_type = 'lstm', # gru
    prediction_length = 30 # 얼마의 기간에 대해서 예측할 것인가
)
predictor = estimator.train(train_ds)

# 예측
forecast_it, ts_it = make_evaluation_predictions(
    dataset = test_ds,
    predictor = predictor,
    num_samples = 10, # number of sample paths we want for evaluation
)

forecasts = list(forecast_it)
tss = list(ts_it)

ts_entry = tss[0]
forecast_entry = forecasts[0] # 0 번째에 prediction한 값이 들어가 있음

# time series modeling error result
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
mean_squared_error(list(forecast_entry.quantile(0.5)), list(ts_entry[0][-30:]), squared=False)